# Supervised Learning Technique 

This notebook show how Supervised Machine Learning techniques can be used to predict the Loan defaulters

Algorithms used:<br>
Logistic regression <br>
Random Forest <br>
Extreme Gradient Boost <br>

Also I have used SMOTE and Random OverSampler to balance the class.

### Load the data

In [1]:
import pandas as pd 
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline 
from AutoClean import AutoClean
import pandas as pd

import sys
import warnings
warnings.filterwarnings("ignore")

# to visualise al the columns in the dataframe
pd.pandas.set_option('display.max_columns', None)

In [2]:
dataset = pd.read_csv('application_train.csv').iloc[:10000,:]
dataset = dataset.dropna()
dataset = pd.get_dummies(dataset)

In [3]:
application_train = dataset

In [6]:
application_train.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 252 entries, 71 to 9992
Data columns (total 210 columns):
 #    Column                                             Non-Null Count  Dtype  
---   ------                                             --------------  -----  
 0    SK_ID_CURR                                         252 non-null    int64  
 1    TARGET                                             252 non-null    int64  
 2    CNT_CHILDREN                                       252 non-null    int64  
 3    AMT_INCOME_TOTAL                                   252 non-null    float64
 4    AMT_CREDIT                                         252 non-null    float64
 5    AMT_ANNUITY                                        252 non-null    float64
 6    AMT_GOODS_PRICE                                    252 non-null    float64
 7    REGION_POPULATION_RELATIVE                         252 non-null    float64
 8    DAYS_BIRTH                                         252 non-null    int64  
 

## Modeling

### Partitioning

In [7]:
#Determine the number of fraud and valid transactions in the dataset

Fraud = application_train[application_train['TARGET']==1]

Valid = application_train[application_train['TARGET']==0]

outlier_fraction = len(Fraud)/float(len(Valid))

print('outlier_fraction for the whole dataset:')
print(outlier_fraction)

print("Fraud Cases : {}".format(len(Fraud)))

print("Valid Cases : {}".format(len(Valid)))

outlier_fraction for the whole dataset:
0.05439330543933055
Fraud Cases : 13
Valid Cases : 239


In [8]:
from sklearn.model_selection import train_test_split

#Create independent and Dependent Features
columns = application_train.columns.tolist()
# Filter the columns to remove data we do not want 
columns = [c for c in columns if c not in ["TARGET"]]
# Remove columns that are categorical type
columns = [c for c in columns if application_train[c].dtype != 'object']
# Remove columns that are categorical type
columns = [c for c in columns if application_train[c].dtype != 'object']
# Store the variable we are predicting 
target = "TARGET"
# Define a random state 
state = np.random.RandomState(42)
X = application_train[columns]
y = application_train[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(201, 209) (201,)
(51, 209) (51,)


### More Feature Selection

using RandomForestClassifier

In [9]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel

In [10]:
application_train

,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,TOTALAREA_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,NAME_CONTRACT_TYPE_Cash loans,NAME_CONTRACT_TYPE_Revolving loans,CODE_GENDER_F,CODE_GENDER_M,FLAG_OWN_CAR_Y,FLAG_OWN_REALTY_N,FLAG_OWN_REALTY_Y,NAME_TYPE_SUITE_Children,NAME_TYPE_SUITE_Family,NAME_TYPE_SUITE_Other_A,NAME_TYPE_SUITE_Other_B,"NAME_TYPE_SUITE_Spouse, partner",NAME_TYPE_SUITE_Unaccompanied,NAME_INCOME_TYPE_Commercial associate,NAME_INCOME_TYPE_State servant,NAME_INCOME_TYPE_Working,NAME_EDUCATION_TYPE_Higher education,NAME_EDUCATION_TYPE_Incomplete higher,NAME_EDUCATION_TYPE_Lower secondary,NAME_EDUCATION_TYPE_Secondary / secondary special,NAME_FAMILY_STATUS_Civil marriage,NAME_FAMILY_STATUS_Married,NAME_FAMILY_STATUS_Separated,NAME_FAMILY_STATUS_Single / not married,NAME_FAMILY_STATUS_Widow,NAME_HOUSING_TYPE_Co-op apartment,NAME_HOUSING_TYPE_House / apartment,NAME_HOUSING_TYPE_Municipal apartment,NAME_HOUSING_TYPE_Office apartment,NAME_HOUSING_TYPE_Rented apartment,NAME_HOUSING_TYPE_With parents,OCCUPATION_TYPE_Accountants,OCCUPATION_TYPE_Cleaning staff,OCCUPATION_TYPE_Cooking staff,OCCUPATION_TYPE_Core staff,OCCUPATION_TYPE_Drivers,OCCUPATION_TYPE_HR staff,OCCUPATION_TYPE_High skill tech staff,OCCUPATION_TYPE_IT staff,OCCUPATION_TYPE_Laborers,OCCUPATION_TYPE_Managers,OCCUPATION_TYPE_Medicine staff,OCCUPATION_TYPE_Private service staff,OCCUPATION_TYPE_Sales staff,OCCUPATION_TYPE_Secretaries,OCCUPATION_TYPE_Security staff,WEEKDAY_APPR_PROCESS_START_FRIDAY,WEEKDAY_APPR_PROCESS_START_MONDAY,WEEKDAY_APPR_PROCESS_START_SATURDAY,WEEKDAY_APPR_PROCESS_START_SUNDAY,WEEKDAY_APPR_PROCESS_START_THURSDAY,WEEKDAY_APPR_PROCESS_START_TUESDAY,WEEKDAY_APPR_PROCESS_START_WEDNESDAY,ORGANIZATION_TYPE_Agriculture,ORGANIZATION_TYPE_Bank,ORGANIZATION_TYPE_Business Entity Type 1,ORGANIZATION_TYPE_Business Entity Type 2,ORGANIZATION_TYPE_Business Entity Type 3,ORGANIZATION_TYPE_Construction,ORGANIZATION_TYPE_Electricity,ORGANIZATION_TYPE_Government,ORGANIZATION_TYPE_Hotel,ORGANIZATION_TYPE_Housing,ORGANIZATION_TYPE_Industry: type 11,ORGANIZATION_TYPE_Industry: type 2,ORGANIZATION_TYPE_Industry: type 7,ORGANIZATION_TYPE_Industry: type 9,ORGANIZATION_TYPE_Kind

In [11]:
X_train.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 201 entries, 337 to 9905
Data columns (total 209 columns):
 #    Column                                             Non-Null Count  Dtype  
---   ------                                             --------------  -----  
 0    SK_ID_CURR                                         201 non-null    int64  
 1    CNT_CHILDREN                                       201 non-null    int64  
 2    AMT_INCOME_TOTAL                                   201 non-null    float64
 3    AMT_CREDIT                                         201 non-null    float64
 4    AMT_ANNUITY                                        201 non-null    float64
 5    AMT_GOODS_PRICE                                    201 non-null    float64
 6    REGION_POPULATION_RELATIVE                         201 non-null    float64
 7    DAYS_BIRTH                                         201 non-null    int64  
 8    DAYS_EMPLOYED                                      201 non-null    int64  


In [91]:
y_train.info(verbose=True)

<class 'pandas.core.series.Series'>
Int64Index: 8000 entries, 8455 to 8310
Series name: TARGET
Non-Null Count  Dtype
--------------  -----
8000 non-null   Int64
dtypes: Int64(1)
memory usage: 132.8 KB


In [12]:
sel = SelectFromModel(RandomForestClassifier(n_estimators = 100, random_state=state))
sel.fit(X_train, y_train)

SelectFromModel(estimator=RandomForestClassifier(random_state=RandomState(MT19937) at 0x2817F5F40))

In [13]:
sel.get_support()

array([ True, False,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True, False, False,  True, False, False, False,
        True,  True, False,  True, False, False, False, False, False,
       False,  True,  True,  True, False,  True,  True,  True,  True,
        True,  True, False,  True,  True,  True,  True,  True, False,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True, False,  True,  True, False,  True,  True,  True,  True,
        True,  True,  True, False,  True, False,  True, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False,  True, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
        True, False, False, False, False, False, False, False,  True,
       False, False,

In [14]:
selected_feat= X_train.columns[(sel.get_support())].tolist()
len(selected_feat)

71

In [15]:
print(selected_feat)

['SK_ID_CURR', 'AMT_INCOME_TOTAL', 'AMT_CREDIT', 'AMT_ANNUITY', 'AMT_GOODS_PRICE', 'REGION_POPULATION_RELATIVE', 'DAYS_BIRTH', 'DAYS_EMPLOYED', 'DAYS_REGISTRATION', 'DAYS_ID_PUBLISH', 'OWN_CAR_AGE', 'FLAG_WORK_PHONE', 'CNT_FAM_MEMBERS', 'REGION_RATING_CLIENT', 'HOUR_APPR_PROCESS_START', 'EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'BASEMENTAREA_AVG', 'YEARS_BEGINEXPLUATATION_AVG', 'YEARS_BUILD_AVG', 'COMMONAREA_AVG', 'ELEVATORS_AVG', 'ENTRANCES_AVG', 'FLOORSMIN_AVG', 'LANDAREA_AVG', 'LIVINGAPARTMENTS_AVG', 'LIVINGAREA_AVG', 'NONLIVINGAPARTMENTS_AVG', 'APARTMENTS_MODE', 'BASEMENTAREA_MODE', 'YEARS_BEGINEXPLUATATION_MODE', 'YEARS_BUILD_MODE', 'COMMONAREA_MODE', 'ELEVATORS_MODE', 'ENTRANCES_MODE', 'FLOORSMAX_MODE', 'FLOORSMIN_MODE', 'LANDAREA_MODE', 'LIVINGAPARTMENTS_MODE', 'LIVINGAREA_MODE', 'NONLIVINGAPARTMENTS_MODE', 'NONLIVINGAREA_MODE', 'APARTMENTS_MEDI', 'BASEMENTAREA_MEDI', 'YEARS_BEGINEXPLUATATION_MEDI', 'YEARS_BUILD_MEDI', 'COMMONAREA_MEDI', 'ENTRANCES_MEDI', 'FLOORSMAX_MEDI',

In [16]:
app_X_train = X_train.copy() #Taking a copy before dropping 
app_X_test = X_test.copy()

In [17]:
X_train = X_train[selected_feat]
X_test = X_test[selected_feat]

In [19]:
print (X_train.shape, X_test.shape)

(201, 71) (51, 71)


------------------------------

## Logistic Regression

In [20]:

from sklearn.linear_model import LogisticRegression

logistic_regressor = LogisticRegression(C = 2)

In [21]:
logistic_regressor.fit(X_train,y_train)


LogisticRegression(C=2)

In [22]:
y_pred = logistic_regressor.predict(X_test)

In [23]:
from sklearn.metrics import classification_report,accuracy_score, f1_score, roc_auc_score
from sklearn.metrics import confusion_matrix

n_errors = (y_pred != y_test).sum()
# Run Classification Metrics
print("{}: {}".format("Logistic Regression errors",n_errors))
print("Accuracy Score :")
print(accuracy_score(y_test,y_pred))
print("Confusion matrix :")
print(confusion_matrix(y_test, y_pred))
print("Classification Report :")
print(classification_report(y_test,y_pred))
print("ROC AUC score is: ",roc_auc_score(y_test,y_pred))

Logistic Regression errors: 4
Accuracy Score :
0.9215686274509803
Confusion matrix :
[[47  0]
 [ 4  0]]
Classification Report :
              precision    recall  f1-score   support

           0       0.92      1.00      0.96        47
           1       0.00      0.00      0.00         4

    accuracy                           0.92        51
   macro avg       0.46      0.50      0.48        51
weighted avg       0.85      0.92      0.88        51

ROC AUC score is:  0.5


## Random Forest - Bagging ensemble of Decision trees

In [24]:
from sklearn.ensemble import RandomForestClassifier

random_forest = RandomForestClassifier(n_estimators = 100, random_state = state, verbose = 1, n_jobs = -1)

In [25]:
random_forest.fit(X_train,y_train)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    0.1s finished


RandomForestClassifier(n_jobs=-1,
                       random_state=RandomState(MT19937) at 0x2817F5F40,
                       verbose=1)

In [26]:
y_pred = random_forest.predict(X_test) 

[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 100 out of 100 | elapsed:    0.0s finished


In [27]:
n_errors = (y_pred != y_test).sum()
# Run Classification Metrics
print("{}: {}".format("Random Forest errors",n_errors))
print("Accuracy Score :")
print(accuracy_score(y_test,y_pred))
print("Confusion matrix :")
print(confusion_matrix(y_test, y_pred))
print("Classification Report :")
print(classification_report(y_test,y_pred))
print("ROC AUC score is: ",roc_auc_score(y_test,y_pred))

Random Forest errors: 4
Accuracy Score :
0.9215686274509803
Confusion matrix :
[[47  0]
 [ 4  0]]
Classification Report :
              precision    recall  f1-score   support

           0       0.92      1.00      0.96        47
           1       0.00      0.00      0.00         4

    accuracy                           0.92        51
   macro avg       0.46      0.50      0.48        51
weighted avg       0.85      0.92      0.88        51

ROC AUC score is:  0.5


## Extreme Gradient Boost Model

In [28]:
from xgboost import XGBClassifier

In [29]:
xgb_classifier = XGBClassifier(n_estimators=100,max_depth=5)

In [30]:
xgb_classifier.fit(X_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=5, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, ...)

In [31]:
y_pred = xgb_classifier.predict(X_test)

In [33]:
n_errors = (y_pred != y_test).sum()
# Run Classification Metrics
print("{}: {}".format("Extreme Gradient Boost errors",n_errors))
print("Accuracy Score :")
print(accuracy_score(y_test,y_pred))
print("Confusion matrix :")
print(confusion_matrix(y_test, y_pred))
print("Classification Report :")
print(classification_report(y_test,y_pred))
print("ROC AUC score is: ",roc_auc_score(y_test,y_pred))

Extreme Gradient Boost errors: 4
Accuracy Score :
0.9215686274509803
Confusion matrix :
[[47  0]
 [ 4  0]]
Classification Report :
              precision    recall  f1-score   support

           0       0.92      1.00      0.96        47
           1       0.00      0.00      0.00         4

    accuracy                           0.92        51
   macro avg       0.46      0.50      0.48        51
weighted avg       0.85      0.92      0.88        51

ROC AUC score is:  0.5


# Balancing class - using SMOTETomek

I have used SMOTETomek which is a method of imblearn. SMOTETomek is a hybrid method which uses an under sampling method (Tomek) in with an over sampling method (SMOTE).

In [34]:
from imblearn.combine import SMOTETomek
from imblearn.under_sampling import NearMiss

In [35]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 201 entries, 337 to 9905
Data columns (total 71 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   SK_ID_CURR                                201 non-null    int64  
 1   AMT_INCOME_TOTAL                          201 non-null    float64
 2   AMT_CREDIT                                201 non-null    float64
 3   AMT_ANNUITY                               201 non-null    float64
 4   AMT_GOODS_PRICE                           201 non-null    float64
 5   REGION_POPULATION_RELATIVE                201 non-null    float64
 6   DAYS_BIRTH                                201 non-null    int64  
 7   DAYS_EMPLOYED                             201 non-null    int64  
 8   DAYS_REGISTRATION                         201 non-null    float64
 9   DAYS_ID_PUBLISH                           201 non-null    int64  
 10  OWN_CAR_AGE                        

In [36]:
# cast to float
y_train = y_train.astype(float)

In [37]:
X_train = X_train.astype(float)

In [38]:
y_train.info()

<class 'pandas.core.series.Series'>
Int64Index: 201 entries, 337 to 9905
Series name: TARGET
Non-Null Count  Dtype  
--------------  -----  
201 non-null    float64
dtypes: float64(1)
memory usage: 3.1 KB


In [39]:
# Implementing Oversampling for Handling Imbalanced 
smk = SMOTETomek(random_state=42)
X_res,y_res=smk.fit_resample(X_train,y_train)

In [40]:
X_res.shape,y_res.shape

((374, 71), (374,))

In [41]:
from collections import Counter
print('Original dataset shape {}'.format(Counter(y_train)))
print('Resampled dataset shape {}'.format(Counter(y_res)))

Original dataset shape Counter({0.0: 192, 1.0: 9})
Resampled dataset shape Counter({0.0: 187, 1.0: 187})


## Logistic regression

In [42]:
logistic_regressor.fit(X_res,y_res)

LogisticRegression(C=2)

In [43]:
y_pred = logistic_regressor.predict(X_test)

In [44]:
n_errors = (y_pred != y_test).sum()
# Run Classification Metrics
print("{}: {}".format("Logistic Regression errors",n_errors))
print("Accuracy Score :")
print(accuracy_score(y_test,y_pred))
print("Confusion matrix :")
print(confusion_matrix(y_test, y_pred))
print("Classification Report :")
print(classification_report(y_test,y_pred))
print("ROC AUC score is: ",roc_auc_score(y_test,y_pred))

Logistic Regression errors: 25
Accuracy Score :
0.5098039215686274
Confusion matrix :
[[24 23]
 [ 2  2]]
Classification Report :
              precision    recall  f1-score   support

           0       0.92      0.51      0.66        47
           1       0.08      0.50      0.14         4

    accuracy                           0.51        51
   macro avg       0.50      0.51      0.40        51
weighted avg       0.86      0.51      0.62        51

ROC AUC score is:  0.5053191489361702


## Random forest

In [45]:
random_forest.fit(X_res,y_res)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    0.1s finished


RandomForestClassifier(n_jobs=-1,
                       random_state=RandomState(MT19937) at 0x2817F5F40,
                       verbose=1)

In [46]:
y_pred = random_forest.predict(X_test)

[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 100 out of 100 | elapsed:    0.0s finished


In [47]:
n_errors = (y_pred != y_test).sum()
# Run Classification Metrics
print("{}: {}".format("Random Forest errors",n_errors))
print("Accuracy Score :")
print(accuracy_score(y_test,y_pred))
print("Confusion matrix :")
print(confusion_matrix(y_test, y_pred))
print("Classification Report :")
print(classification_report(y_test,y_pred))
print("ROC AUC score is: ",roc_auc_score(y_test,y_pred))

Random Forest errors: 4
Accuracy Score :
0.9215686274509803
Confusion matrix :
[[47  0]
 [ 4  0]]
Classification Report :
              precision    recall  f1-score   support

           0       0.92      1.00      0.96        47
           1       0.00      0.00      0.00         4

    accuracy                           0.92        51
   macro avg       0.46      0.50      0.48        51
weighted avg       0.85      0.92      0.88        51

ROC AUC score is:  0.5


## XGBoost

In [48]:
X_res = pd.DataFrame(data=X_res, columns=selected_feat)

In [49]:
xgb_classifier.fit(X_res,y_res)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=5, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, ...)

In [50]:
y_pred = xgb_classifier.predict(X_test)

In [51]:
n_errors = (y_pred != y_test).sum()
# Run Classification Metrics
print("{}: {}".format("Extreme Gradient Boost errors",n_errors))
print("Accuracy Score :")
print(accuracy_score(y_test,y_pred))
print("Confusion matrix :")
print(confusion_matrix(y_test, y_pred))
print("Classification Report :")
print(classification_report(y_test,y_pred))
print("ROC AUC score is: ",roc_auc_score(y_test,y_pred))

Extreme Gradient Boost errors: 4
Accuracy Score :
0.9215686274509803
Confusion matrix :
[[47  0]
 [ 4  0]]
Classification Report :
              precision    recall  f1-score   support

           0       0.92      1.00      0.96        47
           1       0.00      0.00      0.00         4

    accuracy                           0.92        51
   macro avg       0.46      0.50      0.48        51
weighted avg       0.85      0.92      0.88        51

ROC AUC score is:  0.5


# Balancing class - RandomOverSampler 

In [52]:
from imblearn.over_sampling import RandomOverSampler

In [53]:
os =  RandomOverSampler(sampling_strategy=1)

In [54]:
X_train_res, y_train_res = os.fit_resample(X_train, y_train)

In [55]:
X_train_res.shape,y_train_res.shape

((384, 71), (384,))

In [56]:
print('Original dataset shape {}'.format(Counter(y_train)))
print('Resampled dataset shape {}'.format(Counter(y_train_res)))

Original dataset shape Counter({0.0: 192, 1.0: 9})
Resampled dataset shape Counter({0.0: 192, 1.0: 192})


## Logistic regression

In [57]:
logistic_regressor.fit(X_train_res,y_train_res)

LogisticRegression(C=2)

In [58]:
y_pred = logistic_regressor.predict(X_test)

In [59]:
n_errors = (y_pred != y_test).sum()
# Run Classification Metrics
print("{}: {}".format("Logistic Regression errors",n_errors))
print("Accuracy Score :")
print(accuracy_score(y_test,y_pred))
print("Confusion matrix :")
print(confusion_matrix(y_test, y_pred))
print("Classification Report :")
print(classification_report(y_test,y_pred))
print("ROC AUC score is: ",roc_auc_score(y_test,y_pred))

Logistic Regression errors: 27
Accuracy Score :
0.47058823529411764
Confusion matrix :
[[23 24]
 [ 3  1]]
Classification Report :
              precision    recall  f1-score   support

           0       0.88      0.49      0.63        47
           1       0.04      0.25      0.07         4

    accuracy                           0.47        51
   macro avg       0.46      0.37      0.35        51
weighted avg       0.82      0.47      0.59        51

ROC AUC score is:  0.3696808510638298


## Random forest

In [60]:
random_forest.fit(X_train_res,y_train_res)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    0.1s finished


RandomForestClassifier(n_jobs=-1,
                       random_state=RandomState(MT19937) at 0x2817F5F40,
                       verbose=1)

In [61]:
y_pred = random_forest.predict(X_test)

[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 100 out of 100 | elapsed:    0.0s finished


In [62]:
n_errors = (y_pred != y_test).sum()
# Run Classification Metrics
print("{}: {}".format("Random Forest errors",n_errors))
print("Accuracy Score :")
print(accuracy_score(y_test,y_pred))
print("Confusion matrix :")
print(confusion_matrix(y_test, y_pred))
print("Classification Report :")
print(classification_report(y_test,y_pred))
print("ROC AUC score is: ",roc_auc_score(y_test,y_pred))

Random Forest errors: 4
Accuracy Score :
0.9215686274509803
Confusion matrix :
[[47  0]
 [ 4  0]]
Classification Report :
              precision    recall  f1-score   support

           0       0.92      1.00      0.96        47
           1       0.00      0.00      0.00         4

    accuracy                           0.92        51
   macro avg       0.46      0.50      0.48        51
weighted avg       0.85      0.92      0.88        51

ROC AUC score is:  0.5


## XGBoost

In [63]:
X_train_res = pd.DataFrame(data=X_train_res, columns=selected_feat)

In [64]:
X_train_res.shape

(384, 71)

In [65]:
xgb_classifier.fit(X_train_res,y_train_res)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=5, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, ...)

In [66]:
y_pred = xgb_classifier.predict(X_test)

In [67]:
n_errors = (y_pred != y_test).sum()
# Run Classification Metrics
print("{}: {}".format("Extreme Gradient Boost errors",n_errors))
print("Accuracy Score :")
print(accuracy_score(y_test,y_pred))
print("Confusion matrix :")
print(confusion_matrix(y_test, y_pred))
print("Classification Report :")
print(classification_report(y_test,y_pred))
print("ROC AUC score is: ",roc_auc_score(y_test,y_pred))

Extreme Gradient Boost errors: 5
Accuracy Score :
0.9019607843137255
Confusion matrix :
[[46  1]
 [ 4  0]]
Classification Report :
              precision    recall  f1-score   support

           0       0.92      0.98      0.95        47
           1       0.00      0.00      0.00         4

    accuracy                           0.90        51
   macro avg       0.46      0.49      0.47        51
weighted avg       0.85      0.90      0.87        51

ROC AUC score is:  0.48936170212765956
